In [25]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os
import requests

file_name = 'tmdb-movies.csv'
url = f"https://raw.githubusercontent.com/aczzi/similar-movies/refs/heads/main/{file_name}"
response = requests.get(url)

if response.status_code == 200:
    with open(file_name, 'wb') as file:
        file.write(response.content)
else:
    raise Exception('Failed to download file')
file_path = os.path.join(os.getcwd(), file_name)
df = pd.read_csv(file_path)
print("data is loaded")

data is loaded


In [37]:
column_keys = ['original_title','director','keywords','genres'] # vector dimentions

columns = df.columns.tolist()
#cleanning data
for column in columns:
  df[column]=df[column].fillna('')

def combine(row):
  # creating new column in dataframe containing
  return "#".join([str(row[column]) for column in column_keys])

def dataFrame():
  # creating new column in dataframe containing
  return pd.DataFrame(df, columns=column_keys)

def movie_dataframe(index):
  return pd.DataFrame(df[df.index==index], columns=column_keys)

def get_movie_summary(index):
  ldf = movie_dataframe(index)
  if ldf.empty:
    raise Exception('Movie not found')

  return " ".join([f"{column}: {ldf[column].values[0]}\n" for column in column_keys])

def get_index_from_title(title):
    ldf = df['original_title'] == title
    if ldf.empty:
      raise Exception('Movie not found')
    try:
      return df[ldf].index[0]
    except:
      raise Exception('Movie not found')

df['combined']=df.apply(combine,axis=1)
vector=CountVectorizer()
matrix=vector.fit_transform(df['combined'])    #method to vectorize the text data
#print("vectorize")
#print(matrix)
cosine=cosine_similarity(matrix)               #applying the function
#print("cosine_similarity")
#print(cosine)

In [38]:
print("Choose a movie title:")
a = input()

movie_index=get_index_from_title(a)
similar=list(enumerate(cosine[movie_index]))
sorted_movies=sorted(similar, key=lambda x:x[1],reverse=True)  #sorting
#Top ten movies
print(get_movie_summary(sorted_movies[0][0]))
print('Similar movies found:')
for movie in sorted_movies[1:10]:
  print(get_movie_summary(movie[0]))

Choose a movie title:
Alien
original_title: Alien
 director: Ridley Scott
 keywords: android|self-destruction|countdown|space marine|space suit
 genres: Horror|Action|Thriller|Science Fiction

Similar movies found:
original_title: Aliens
 director: James Cameron
 keywords: android|extraterrestrial technology|space marine|spaceman|cryogenics
 genres: Horror|Action|Thriller|Science Fiction

original_title: Moonraker
 director: Lewis Gilbert
 keywords: venice|mass murder|space marine|space suit|marcus square
 genres: Action|Adventure|Thriller|Science Fiction

original_title: Lifeforce
 director: Tobe Hooper
 keywords: space marine|vampire|flying saucer|comet|alien
 genres: Action|Science Fiction|Thriller|Horror|Fantasy

original_title: Planet of the Apes
 director: Tim Burton
 keywords: gorilla|space marine|space suit|revolution|chimp
 genres: Thriller|Science Fiction|Action|Adventure

original_title: AlienÂ³
 director: David Fincher
 keywords: prison|android|spacecraft|space marine|impri